In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [10]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'UJIndoorLoc-model'
id_name = ''
target = 'LONGITUDE'
drop_col = ['USERID', 'PHONEID', 'TIMESTAMP']

file_name = "UJIndoorLoc_test.csv"
test_file_name = "UJIndoorLoc_test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1)
df = df.drop(drop_col,axis=1)
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

   WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     100     100     100     100   
3     100     100     100     100     100     100     100     100     100   
4     100     100     100     100     100     100     100     100     100   

   WAP010  ...  WAP516  WAP517  WAP518  WAP519  WAP520      LATITUDE  FLOOR  \
0     100  ...     100     100     100     100     100  4.864890e+06      1   
1     100  ...     100     100     100     100     100  4.864840e+06      4   
2     100  ...     100     100     100     100     100  4.864847e+06      4   
3     100  ...     100     100     100     100     100  4.864843e+06      4   
4     100  ...     100     100     100     100     100  4.864922e+06      2   

   BUILDINGID  SPACEID  RELATIVEPOSITION  
0           1      

In [12]:
f = open(test_file_name)
count = 0
pred = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    pred.append(float(response))
    count = count + 1


In [8]:
print(pred[:20])

import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred)
print("test mse = {}".format(mse))

[1998.524658203125, 2002.6065673828125, 2004.369873046875, 2004.9896240234375, 1999.8597412109375, 2003.556640625, 1999.30517578125, 2003.9404296875, 1994.491943359375, 2003.3623046875, 2003.67822265625, 1990.9012451171875, 1997.35302734375, 1988.655029296875, 2002.707275390625, 2001.2862548828125, 1993.7510986328125, 2005.1400146484375, 2003.60107421875, 1979.8896484375]
test mse = 75.61080610368741


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

FileNotFoundError: [Errno 2] File  does not exist: ''

In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [5]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

NameError: name 'kaggle_pred_df' is not defined

In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}